## Import library

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc

## Read Dataset

In [2]:
a = pd.read_csv("/home/quan/PROJECT/Machine Learning with Biomedical Signals/kaggle_downloads/user_a.csv")
print(a.head(3))
print(a.shape)
print(a.info())

   Class          AF3        AF3.1     AF3.2     AF3.3     AF3.4     AF3.5  \
0    1.0  3569.164550  2063.892754  1.673726  4.444736  0.526209  3.002088   
1    1.0  3568.423670  2063.099248  1.897790  3.728823  1.304186  1.854353   
2    1.0  3568.157929  2062.445859  2.798014  2.574504  1.120537  1.958819   

      AF3.6     AF3.7           F7  ...       F8.6       F8.7          AF4  \
0  1.425022  3.302739  3563.803888  ...  45.468326  72.508750  3701.186330   
1  1.366575  2.546458  3563.560922  ...  36.551948  66.931186  3725.210509   
2  0.982433  2.258622  3563.279981  ...  40.754308  66.816547  3724.417296   

         AF4.1      AF4.2      AF4.3      AF4.4      AF4.5      AF4.6  \
0  2182.676835  18.192418  41.349662  16.004756  42.046467  46.280843   
1  2180.197439   8.820788  38.012788  19.601233  29.431054  38.559351   
2  2176.823208  18.159202  23.612639  14.378291  19.555084  43.210004   

       AF4.7  
0  73.565719  
1  67.470041  
2  67.781924  

[3 rows x 113 column

In [3]:
print(a.columns[1])

AF3


In [19]:
a['Class'].unique()

array([1., 2., 0.])

## Splitting the Features and Target

In [4]:
X = a.drop(columns =['Class'])
y = a['Class']

In [5]:
X.head()

,AF3,AF3.1,AF3.2,AF3.3,AF3.4,AF3.5,AF3.6,AF3.7,F7,F7.1,...,F8.6,F8.7,AF4,AF4.1,AF4.2,AF4.3,AF4.4,AF4.5,AF4.6,AF4.7
0,3569.164550,2063.892754,1.673726,4.444736,0.526209,3.002088,1.425022,3.302739,3563.803888,2060.239057,...,45.468326,72.508750,3701.186330,2182.676835,18.192418,41.349662,16.004756,42.046467,46.280843,73.565719
1,3568.423670,2063.099248,1.897790,3.728823,1.304186,1.854353,1.366575,2.546458,3563.560922,2059.969372,...,36.551948,66.931186,3725.210509,2180.197439,8.820788,38.012788,19.601233,29.431054,38.559351,67.470041
2,3568.157929,2062.445859,2.798014,2.574504,1.120537,1.958819,0.982433,2.258622,3563.279981,2059.543651,...,40.754308,66.816547,3724.417296,2176.823208,18.159202,23.612639,14.378291,19.555084,43.210004,67.781924
3,3567.710021,2062.112673,2.181775,3.610507,0.629608,2.155876,0.856275,2.233711,3562.787801,2059.317489,...,38.074628,63.915386,3725.822160,2177.089059,19.737616,29.484396,15.793034,25.713513,39.250246,65.031031
4,3565.546124,2063.128867,1.685161,3.384311,0.677526,1.795798,0.927924,1.909810,3562.655091,2059.139105,...,35.357384,64.534645,3723.053978,2167.798335,8.429414,26.374975,14.920736,35.675266,33.901687,66.956313


In [6]:
y.head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: Class, dtype: float64

## Feature Extraction

In [7]:
# def extract_features(X_raw):
#     X_extracted = []
#     n_channels = 14 # Assuming there are 14 channels
#     values_per_channel = 8

#     for i in range(X_raw.shape[0]): 
#         row = X_raw.iloc[i].values
#         features = []
#         for j in range(n_channels):
#             segment = row[j * values_per_channel : (j + 1) * values_per_channel]
#             features.append(np.mean(segment))      # Mean
#             features.append(np.std(segment))       # Std
#             features.append(np.sum(segment**2))    # Energy
#         X_extracted.append(features)

#     return np.array(X_extracted)
# X_extracted = extract_features(a)

## Split Train/Test

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Standardize the extracted features

In [9]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)

## Model Selection

### 1. Comparing the models with default hyperparameter using Cross Validation

In [10]:
models = [LogisticRegression(max_iter = 10000),
            SVC(kernel = 'linear'),
            DecisionTreeClassifier(),
            RandomForestClassifier(random_state=0),
            KNeighborsClassifier()]

def compare_models_cv():
    for model in models:
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
        mean_cv_score = round(cv_scores.mean() * 100, 2)
        model.fit(X_train_scaled, y_train)
        test_score = round(model.score(X_test_scaled, y_test) * 100, 2)

        # Print the results
        print(f"Model: {model}")
        print(f"  Cross-validation accuracies: {cv_scores}")
        print(f"  Mean CV Accuracy: {mean_cv_score}%")
        print(f"  Test Accuracy: {test_score}%")
        print("----------------------------------")

compare_models_cv()


Model: LogisticRegression(max_iter=10000)
  Cross-validation accuracies: [0.61171367 0.62906725 0.63774403 0.63557484 0.59130435]
  Mean CV Accuracy: 62.11%
  Test Accuracy: 63.19%
----------------------------------
Model: SVC(kernel='linear')
  Cross-validation accuracies: [0.62039046 0.60737527 0.63557484 0.63557484 0.62826087]
  Mean CV Accuracy: 62.54%
  Test Accuracy: 62.85%
----------------------------------
Model: DecisionTreeClassifier()
  Cross-validation accuracies: [0.70932755 0.69848156 0.69631236 0.63774403 0.68043478]
  Mean CV Accuracy: 68.45%
  Test Accuracy: 71.01%
----------------------------------
Model: RandomForestClassifier(random_state=0)
  Cross-validation accuracies: [0.87201735 0.84598698 0.88503254 0.85466377 0.84565217]
  Mean CV Accuracy: 86.07%
  Test Accuracy: 88.02%
----------------------------------
Model: KNeighborsClassifier()
  Cross-validation accuracies: [0.84598698 0.81561822 0.86334056 0.85032538 0.84130435]
  Mean CV Accuracy: 84.33%
  Test Accu

- RandomForest Classifier has the highest accuracy value with default hyperparameter values

### 2. Compare the models with different Hyperparameter values using GridSearchCV

In [11]:

models = [LogisticRegression(max_iter = 10000),
            SVC(kernel = 'linear'),
            DecisionTreeClassifier(),
            RandomForestClassifier(random_state=0),
            KNeighborsClassifier()]

model_hyperparameter = {
    'log_reg_hyperparameters': {'C': [1, 5, 10, 20, 100]},
    'SVC_hyperparameters': {'C': [1, 5, 10, 20, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
    'DecisionTree_hyperparameters': {
        'max_depth': [3, 5, 10, 20, None],
        'min_samples_split': [2, 5, 10]
    },
    'RandomForest_hyperparameters': {'n_estimators': [10, 20,  50, 100]},
    'KNeighbors_hyperparameters': {'n_neighbors': [3, 5, 7, 9, 10]},
}   

def compare_models_gridsearchCV(list_of_models, model_hyperparameter):
    results = []
    model_keys = list(model_hyperparameter.keys())

    for i, model in enumerate(list_of_models):
        key = model_keys[i]
        params = model_hyperparameter[key]

        try:
            classifier = GridSearchCV(model, params, cv=5, error_score=np.nan)
            classifier.fit(X_train_scaled, y_train)
            best_model = classifier.best_estimator_

            test_accuracy = best_model.score(X_test_scaled, y_test)
            best_score = classifier.best_score_

        except Exception as e:
            print(f"⚠️ Error for model {model}: {e}")
            best_score = np.nan
            test_accuracy = np.nan
            best_model = None

        results.append({
            'model used': str(model),
            'highest CV score': None if pd.isna(best_score) else round(best_score * 100, 2),
            'test accuracy': None if pd.isna(test_accuracy) else round(test_accuracy * 100, 2),
            'best parameters': classifier.best_params_ if best_model is not None else None
        })

    return pd.DataFrame(results)
compare_models_gridsearchCV(models, model_hyperparameter)

,model used,highest CV score,test accuracy,best parameters
0,LogisticRegression(max_iter=10000),62.45,64.06,{'C': 20}
1,SVC(kernel='linear'),88.19,90.97,"{'C': 20, 'kernel': 'rbf'}"
2,DecisionTreeClassifier(),69.44,70.14,"{'max_depth': 20, 'min_samples_split': 2}"
3,RandomForestClassifier(random_state=0),86.07,88.02,{'n_estimators': 100}
4,KNeighborsClassifier(),86.67,88.02,{'n_neighbors': 3}


## Conclusion

- After evaluating multiple classification models using both manual cross-validation and GridSearchCV, the `Support Vector Classifier (SVC)` with the RBF kernel emerged as the most promising choice. It consistently achieved **the highest cross-validation accuracy (up to 88.19%)** and an impressive **test accuracy of 90.97%**, outperforming all other models in terms of generalization performance.

- The RBF kernel enables the SVC to capture non-linear relationships between features, which is particularly beneficial in complex classification tasks where class boundaries are not linearly separable. Compared to simpler models such as Logistic Regression and Decision Trees, the RBF-SVC demonstrated superior robustness and stability across different data splits.

- Additionally, hyperparameter tuning via GridSearchCV identified optimal settings (`C=20`, `kernel='rbf'`) that significantly enhanced performance. This result confirms the model’s strong fit to the data without signs of overfitting, as evidenced by its high accuracy on both validation and test sets.

**Conclusion:** Based on both cross-validation results and final test accuracy, the RBF-kernel SVC is selected as the best-performing model for this classification task.

